In [1]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

import time
import datetime

import scipy.stats as sts

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_json('table.jsonl', lines=True)

Gold = a * time + b

In [3]:
Y, X = data[data['type']=='interval'].dropna(subset=[['gold','time']])['gold'].as_matrix().reshape(-1, 1), data[data['type']=='interval'].dropna(subset=[['gold','time']])['time'].as_matrix().reshape(-1, 1)

In [4]:
def smodt(X,Y):
    X = sm.add_constant(X)
    mod = sm.OLS(Y, X)
    res = mod.fit()
    print('\n', res.summary())
    
def Testing_grid_reg(X_train, Y_train):
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    grid = {'fit_intercept':[True,False], 
            'normalize':[True,False], 
            'copy_X':[True, False]}
    
    clf = LinearRegression(n_jobs = 5)
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='r2', param_grid=grid, cv=cv, return_train_score=True)
    gs.fit(X_train,Y_train)
    
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    lm = gs.best_estimator_
    
    lm.fit(X,Y)
    params = np.append(lm.intercept_,lm.coef_)
    predictions = lm.predict(X)
    
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((X-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-sts.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]
    
    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)
    
    myDF3 = pd.DataFrame()
    myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [params,sd_b,ts_b,p_values]
    
    print ('\n', myDF3)
    
    smodt(X,Y)
    return gs.best_estimator_ 

Значимы​​ ли​​ коэффициенты​​ регрессии?​​ Нужно​​ ли​​ использовать​​ intercept​​ в
данной​​ модели?​​ Какой​​ смысл​​ в​​ терминах​​ игры​​ имеет​​ коэффициент​​ наклона?

In [5]:
clf = Testing_grid_reg(X,Y)

Time elapsed: 0:00:00.855049
0.716 (+/-0.005) for {'copy_X': True, 'fit_intercept': True, 'normalize': True}
0.716 (+/-0.005) for {'copy_X': True, 'fit_intercept': True, 'normalize': False}
0.636 (+/-0.004) for {'copy_X': True, 'fit_intercept': False, 'normalize': True}
0.636 (+/-0.004) for {'copy_X': True, 'fit_intercept': False, 'normalize': False}
0.716 (+/-0.005) for {'copy_X': False, 'fit_intercept': True, 'normalize': True}
0.716 (+/-0.005) for {'copy_X': False, 'fit_intercept': True, 'normalize': False}
0.636 (+/-0.004) for {'copy_X': False, 'fit_intercept': False, 'normalize': True}
0.636 (+/-0.004) for {'copy_X': False, 'fit_intercept': False, 'normalize': False}

    Coefficients  Standard Errors  t values  Probabilites
0    -3867.9249           50.486   -76.614           0.0
1        5.8809            0.026   229.697           0.0

                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:               

Оба коэффициента значимы - $|t_{value}| > P_{value} $. 

В начале игры(time = 0), золото(intercept) будет равно -3867.9249, что не отражает реальность, а также говорит о наличии других проблем (например, мультикорреляция). 

Коэффициент наклона говорит нам, что увеличение золота на одну единицу приводит к увеличению времени на 5.8809 сек, или каждые 5.8809 секунд приносят игроку 1 голд.

In [8]:
data[data['type']=='DOTA_COMBATLOG_DEATH']

,assists,attackerhero,attackerillusion,attackername,attackerteam,camps_stacked,creeps_stacked,deaths,denies,ehandle,...,unit_list,value,valuename,vec_x,vec_y,x,xp,xp_reason,y,z
83671,NaN,0.0,0.0,npc_dota_earth_spirit_stone,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92754,NaN,0.0,0.0,npc_dota_earth_spirit_stone,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98731,NaN,1.0,0.0,npc_dota_hero_tinker,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99486,NaN,1.0,0.0,npc_dota_hero_silencer,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101596,NaN,1.0,0.0,npc_dota_hero_tinker,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102792,NaN,1.0,0.0,npc_dota_hero_terrorblade,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103352,NaN,1.0,0.0,npc_dota_hero_tinker,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103478,NaN,1.0,0.0,npc_dota_hero_luna,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105067,NaN,0.0,0.0,npc_dota_creep_badguys_ranged,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105352,NaN,1.0,0.0,npc_dota_hero_luna,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
